# Kiwi와 MeCab을 이용한 한국어 토크나이징

In [27]:
import pandas as pd
from konlpy.tag import Mecab
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

In [28]:
# df = pd.read_csv('../extract/dcinside/result/contents_of_제네시스_in_car_new1.csv')
df = pd.read_csv('../extract/dcinside/result/contents_of_아이오닉_in_car_new1.csv')
# df = pd.read_csv('../extract/dcinside/result/merged_naver_cafe_data.csv')
# df = pd.read_csv('../extract/dcinside/result/bobaedream_아이오닉_Extract_Result.csv')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Title'] = df['Title'].fillna('')
df['Body'] = df['Body'].fillna('')
df['Comment'] = df['Comment'].fillna('')
df.head()

,Title,Date,Time,Body,Comment,View,Like,Community,Url,Num_comments,DC_app,Dislike
0,아이오닉5롱레나 GV60오너잇음?,2022-01-08,19:12,없겟지..후 궁금한게잇다구,,60,0.0,dcinside,https://gall.dcinside.com/board/view/?id=car_n...,0,False,0.0
1,아이오닉은 이 색상이 갑임?,2022-01-08,20:02,루시드블루펄 이라는데 저게 갑임?,,101,0.0,dcinside,https://gall.dcinside.com/board/view/?id=car_n...,0,False,0.0
2,GV60 스탠다드 후륜견적 7100이면 어떠냐,2022-01-08,20:50,그돈씨 외제차 얘기하지말고\n아이오닉 롱레 프레스티지 vs 60 스탠 후륜\n어느게 낫냐,60,110,0.0,dcinside,https://gall.dcinside.com/board/view/?id=car_n...,1,False,0.0
3,아이오닉 니로 이런거 타는애들이 도요타 피꺼솟한다ㅋ,2022-01-09,00:18,,애초에 흉기는 싼마이에 쓰고 가성비 내장 쓰는것.\n지팔공부터 고민대상이지.\n그정...,230,4.0,dcinside,https://gall.dcinside.com/board/view/?id=car_n...,6,False,0.0
4,아이오닉이 포르쉐 이기냐?,2022-01-09,21:13,전기차가 그리 빠르냐?,,85,0.0,dcinside,https://gall.dcinside.com/board/view/?id=car_n...,3,False,1.0


## MeCab 토크나이저
DC인사이드 아이노닉에 대해 1초 이내

In [29]:
# https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit?gid=589544265#gid=589544265
mecab = Mecab()

def get_nouns_mecab(text):
    if not isinstance(text, str):
        return []
    
    tokens = mecab.pos(text)
    nouns = [word for word, pos in tokens if pos.startswith('NNG') or pos.startswith('NNP') or pos.startswith('SL')]
    return list(set(nouns))

## Kiwi 토크나이저
DC인사이드 아이오닉에 대해 28초 소요

In [106]:
# https://github.com/bab2min/kiwipiepy
kiwi = Kiwi()
stopwords = Stopwords()

# 사용자 단어 추가
USER_WORD = [
    '현대자동차', '기아', '현차', '현대차', '현기', '현대', '', '', 
    '아이오닉', '제네시스', '아반떼', '니로', '벤츠', '토레스', '쏘나타', '카니발', '하이브리드', '모닝', '봉고', '테슬라', '포르쉐', '레이', '쏘렌토', '르노', '렉서스', 
]
for word in USER_WORD:
    kiwi.add_user_word(word, 'NNP')  # 고유명사로 추가

# 불용어 리스트 생성 및 추가
USER_STOP_WORD = [
    '현대자동차', '기아', '현차', '현대차', '모델', '전기차', '자동차', '중고', '차', '현기', '내연', '운전', '현대', '차종', '신차', '국산', '전기', '', '', 
    '아이오닉', '제네시스', '아반떼', '니로', '벤츠', '토레스', '쏘나타', '카니발', '하이브리드', '모닝', '봉고', '테슬라', '포르쉐', '레이', '쏘렌토', '르노', '렉서스', 

    '차이', '돈', '밥', '이유', '글', '집', '세대', '고민', '내년', '올해', '비교', '개', '값', '닉', '급', '생각', '뒤', '기준', '현재', '사람', '애', '카',
    '회사', '키', '코', '가능', '시작', '오늘', '사진', '봄', '차량', '정도', '기본', '탐', '일반', '브랜드', '막', '나라', '택', '풀',
    '앞', '뒤', '추천', '말', '맘',  '땅', '주변', '추가', '신', '최고', '기사', '길', '순서', '도로', '초반', '기차', '미래', '배', '세계', '일', '자체',
    '테', '독', '상실', '만약', '필요', '베', '느낌', '렌', '랜', '지금', '이번', '노인', '페리', '형', '다시', '렁', '듬', '져', '인', '전', '쌈', '때', '트', '감상', 
    '따리', '뉴', '점', '특유', '동안', '엔', '칸', '특징', '인생', '정보', '예정', '롱', '주', '다음', '시속', '르', '뉘', '쉐', '츄', '쌍', '줄', '혼', '', '', '', '', 
    '좆', '병신', '시발', '지랄', '새끼', '색', '존나', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 
]
for stop_word in USER_STOP_WORD:
    stopwords.add((stop_word, 'NNG'))
    
# 명사 추출 함수
def get_nouns_kiwi(sentence):
    tokens = kiwi.tokenize(sentence, stopwords=stopwords, normalize_coda=True)
    # 명사(NNG, NNP), 동사(VV), 형용사(VA)
    filtered_tokens = [token.form for token in tokens if token.tag in {'NNG', 'NNP'}]
    return filtered_tokens

In [107]:
tokenizer = get_nouns_mecab

# 게시글 제목과 내용을 합친 후 명사 추출
TITLE_WEIGHT = 5
BODY_WEIGHT = 2
COMMENT_WEIGHT = 0
df['nouns_title'] = df['Title'].apply(tokenizer)
df['nouns_body'] = df['Body'].apply(tokenizer)
df['nouns_comment'] = df['Comment'].apply(tokenizer)
df['nouns'] = TITLE_WEIGHT * df['nouns_title'] + BODY_WEIGHT * df['nouns_body'] + COMMENT_WEIGHT * df['nouns_comment']
df[['Date', 'Title', 'Body', 'nouns']].head()

,Date,Title,Body,nouns
0,2022-01-08,아이오닉5롱레나 GV60오너잇음?,없겟지..후 궁금한게잇다구,"[레나, GV, 아이오닉, 롱, 오너, 레나, GV, 아이오닉, 롱, 오너, 레나,..."
1,2022-01-08,아이오닉은 이 색상이 갑임?,루시드블루펄 이라는데 저게 갑임?,"[갑, 아이오닉, 색상, 갑, 아이오닉, 색상, 갑, 아이오닉, 색상, 갑, 아이오..."
2,2022-01-08,GV60 스탠다드 후륜견적 7100이면 어떠냐,그돈씨 외제차 얘기하지말고\n아이오닉 롱레 프레스티지 vs 60 스탠 후륜\n어느게 낫냐,"[견적, 스탠다드, GV, 후륜, 견적, 스탠다드, GV, 후륜, 견적, 스탠다드,..."
3,2022-01-09,아이오닉 니로 이런거 타는애들이 도요타 피꺼솟한다ㅋ,,"[니로, 솟, 아이오닉, 도요타, 애, 피, 니로, 솟, 아이오닉, 도요타, 애, ..."
4,2022-01-09,아이오닉이 포르쉐 이기냐?,전기차가 그리 빠르냐?,"[아이오닉, 포르쉐, 아이오닉, 포르쉐, 아이오닉, 포르쉐, 아이오닉, 포르쉐, 아..."
